In [1]:
import psycopg2
import pandas as pd
from decouple import config
# # https://gist.github.com/rg3915/5fb3a2e7338115bc92e82b7a9a2b372b

In [2]:
df_produtos = pd.read_excel('produtos-com-categoria.xlsx')
df_categorias = pd.read_excel('categorias.xlsx')

In [3]:
try:
    connection = psycopg2.connect(user = "my_user",
                                  password = config("PASSWORD"),
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "my_db")

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")

    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)

{'user': 'my_usuario', 'dbname': 'my_db', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 11.9 (Ubuntu 11.9-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-10ubuntu2) 9.3.0, 64-bit',) 



In [4]:
cur = connection.cursor()

In [5]:
# https://gist.github.com/rg3915/0f63ee9bde818c4a56abb110c94b855b
def get_data(items, field='categoria'):
    '''
    Lê os dados para extrair id e um outro campo como identificador único
    e monta um dicionário.
    '''
    my_dict = {}
    for item in items:
        my_dict[str(item['id'])] = item[field]
    return my_dict

In [6]:
items = df_categorias.T.apply(dict).tolist()
items

[{'id': 101, 'categoria': 'bebidas'},
 {'id': 210, 'categoria': 'sobremesas'},
 {'id': 314, 'categoria': 'cereais'},
 {'id': 420, 'categoria': 'frios'},
 {'id': 598, 'categoria': 'limpeza'},
 {'id': 670, 'categoria': 'hortifruti'},
 {'id': 700, 'categoria': 'perfumaria'},
 {'id': 842, 'categoria': 'mercearia'},
 {'id': 915, 'categoria': 'bazar'},
 {'id': 1000, 'categoria': 'pet shop'}]

In [7]:
d = {}
for item in items:
    print(item)
    d[str(item['id'])] = item['categoria']

{'id': 101, 'categoria': 'bebidas'}
{'id': 210, 'categoria': 'sobremesas'}
{'id': 314, 'categoria': 'cereais'}
{'id': 420, 'categoria': 'frios'}
{'id': 598, 'categoria': 'limpeza'}
{'id': 670, 'categoria': 'hortifruti'}
{'id': 700, 'categoria': 'perfumaria'}
{'id': 842, 'categoria': 'mercearia'}
{'id': 915, 'categoria': 'bazar'}
{'id': 1000, 'categoria': 'pet shop'}


In [8]:
dict_categoria = get_data(df_categorias.T.apply(dict).tolist())
dict_categoria

{'101': 'bebidas',
 '210': 'sobremesas',
 '314': 'cereais',
 '420': 'frios',
 '598': 'limpeza',
 '670': 'hortifruti',
 '700': 'perfumaria',
 '842': 'mercearia',
 '915': 'bazar',
 '1000': 'pet shop'}

In [9]:
for produto in df_produtos.head().iterrows():
    print(produto[1][2])

2.59
10.47
10.47
10.47
10.47


In [10]:
for produto in df_produtos.head().itertuples():
    print(produto.produto, produto.preco, produto.categoria)

uva 2.59 670
Suco de uva 10.47 101
Suco de manga 10.47 101
Suco de maça 10.47 101
Suco de laranja 10.47 101


In [11]:
for produto in df_produtos.itertuples():
    campo = dict_categoria.get(str(produto.categoria))
    print(produto.categoria, campo)
    print({'categoria': campo})
    cur.execute(f"SELECT * FROM categoria WHERE nome='{campo}';")
    categoria = cur.fetchall()
    print(categoria)
    print(categoria[0])
    print(categoria[0][0])
    print()

670 hortifruti
{'categoria': 'hortifruti'}
[(6, 'hortifruti')]
(6, 'hortifruti')
6

101 bebidas
{'categoria': 'bebidas'}
[(1, 'bebidas')]
(1, 'bebidas')
1

101 bebidas
{'categoria': 'bebidas'}
[(1, 'bebidas')]
(1, 'bebidas')
1

101 bebidas
{'categoria': 'bebidas'}
[(1, 'bebidas')]
(1, 'bebidas')
1

101 bebidas
{'categoria': 'bebidas'}
[(1, 'bebidas')]
(1, 'bebidas')
1

700 perfumaria
{'categoria': 'perfumaria'}
[(7, 'perfumaria')]
(7, 'perfumaria')
7

842 mercearia
{'categoria': 'mercearia'}
[(8, 'mercearia')]
(8, 'mercearia')
8

915 bazar
{'categoria': 'bazar'}
[(9, 'bazar')]
(9, 'bazar')
9

700 perfumaria
{'categoria': 'perfumaria'}
[(7, 'perfumaria')]
(7, 'perfumaria')
7

598 limpeza
{'categoria': 'limpeza'}
[(5, 'limpeza')]
(5, 'limpeza')
5

598 limpeza
{'categoria': 'limpeza'}
[(5, 'limpeza')]
(5, 'limpeza')
5

1000 pet shop
{'categoria': 'pet shop'}
[(10, 'pet shop')]
(10, 'pet shop')
10

1000 pet shop
{'categoria': 'pet shop'}
[(10, 'pet shop')]
(10, 'pet shop')
10

314 cereais


In [12]:
cur.execute("DELETE FROM produtos_categoria;")
connection.commit()

In [13]:
for produto in df_produtos.itertuples():
    campo = dict_categoria.get(str(produto.categoria))
    cur.execute(f"SELECT * FROM categoria WHERE nome='{campo}';")
    categoria = cur.fetchall()
    categoria_id = categoria[0][0]
    # print(categoria_id)
    nome_produto = produto.produto
    valor_produto = produto.preco
    categoria_tipo = int(categoria_id)
    cur.execute(f"INSERT INTO produtos_categoria (nome_produto, valor_produto, categoria_tipo) VALUES ('{nome_produto}',{valor_produto},{categoria_tipo})")

In [14]:
connection.commit()

In [15]:
cur.execute("SELECT * FROM produtos_categoria;")
cur.fetchall()

[(1, 'uva', Decimal('2.59'), 6),
 (2, 'Suco de uva', Decimal('10.47'), 1),
 (3, 'Suco de manga', Decimal('10.47'), 1),
 (4, 'Suco de maça', Decimal('10.47'), 1),
 (5, 'Suco de laranja', Decimal('10.47'), 1),
 (6, 'shampoo', Decimal('3.97'), 7),
 (7, 'sardinha enlatada', Decimal('8.63'), 8),
 (8, 'sandália Havaianas', Decimal('4.56'), 9),
 (9, 'sabonete', Decimal('2.75'), 7),
 (10, 'Sabão em pó', Decimal('17.93'), 5),
 (11, 'Sabão em barra', Decimal('1.75'), 5),
 (12, 'Ração para gato', Decimal('14.96'), 10),
 (13, 'Ração para cachorro', Decimal('8.63'), 10),
 (14, 'quinoa', Decimal('7.12'), 3),
 (15, 'Queijo mussarela', Decimal('5.33'), 4),
 (16, 'Queijo frescal', Decimal('5.55'), 4),
 (17, 'Pudim', Decimal('2.55'), 2),
 (18, 'Petit Gateau', Decimal('6.77'), 2),
 (19, 'óleo de soja', Decimal('4.6'), 8),
 (20, 'óleo de girassol', Decimal('16.85'), 8),
 (21, 'mortadela', Decimal('6.39'), 4),
 (22, 'melão', Decimal('1.12'), 6),
 (23, 'melancia', Decimal('2.1'), 6),
 (24, 'manga', Decimal(

In [16]:
cur.execute("SELECT * FROM produtos_categoria as p INNER JOIN categoria ON (p.categoria_tipo = categoria.id) ORDER BY p.id;")
result = cur.fetchall()
result

[(1, 'uva', Decimal('2.59'), 6, 6, 'hortifruti'),
 (2, 'Suco de uva', Decimal('10.47'), 1, 1, 'bebidas'),
 (3, 'Suco de manga', Decimal('10.47'), 1, 1, 'bebidas'),
 (4, 'Suco de maça', Decimal('10.47'), 1, 1, 'bebidas'),
 (5, 'Suco de laranja', Decimal('10.47'), 1, 1, 'bebidas'),
 (6, 'shampoo', Decimal('3.97'), 7, 7, 'perfumaria'),
 (7, 'sardinha enlatada', Decimal('8.63'), 8, 8, 'mercearia'),
 (8, 'sandália Havaianas', Decimal('4.56'), 9, 9, 'bazar'),
 (9, 'sabonete', Decimal('2.75'), 7, 7, 'perfumaria'),
 (10, 'Sabão em pó', Decimal('17.93'), 5, 5, 'limpeza'),
 (11, 'Sabão em barra', Decimal('1.75'), 5, 5, 'limpeza'),
 (12, 'Ração para gato', Decimal('14.96'), 10, 10, 'pet shop'),
 (13, 'Ração para cachorro', Decimal('8.63'), 10, 10, 'pet shop'),
 (14, 'quinoa', Decimal('7.12'), 3, 3, 'cereais'),
 (15, 'Queijo mussarela', Decimal('5.33'), 4, 4, 'frios'),
 (16, 'Queijo frescal', Decimal('5.55'), 4, 4, 'frios'),
 (17, 'Pudim', Decimal('2.55'), 2, 2, 'sobremesas'),
 (18, 'Petit Gateau'

In [17]:
pd.DataFrame(result, columns = ['id', 'nome_produto', 'valor_produto', 'categoria_tipo', 'id', 'nome'])

,id,nome_produto,valor_produto,categoria_tipo,id,nome
0,1,uva,2.59,6,6,hortifruti
1,2,Suco de uva,10.47,1,1,bebidas
2,3,Suco de manga,10.47,1,1,bebidas
3,4,Suco de maça,10.47,1,1,bebidas
4,5,Suco de laranja,10.47,1,1,bebidas
...,...,...,...,...,...,...
59,60,Apresuntado,4.82,4,4,frios
60,61,Ambrosia,2.55,2,2,sobremesas
61,62,Água sanitária,13.89,5,5,limpeza
62,63,abacaxi,1.02,6,6,hortifruti
